In [1]:
"""
Decompose steering direction into SAE features.

Find which SAE features (from llama_scope) are most aligned with
the self/other MMS steering direction at layer 21.
"""
import json
import torch
import numpy as np
from sae_lens import SAE


In [2]:
TOP_K = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

for version in ["V1", "V3"]:
    for layer in range(1, 25):
        print(f"Processing layer {layer} for version {version}")
        DIRECTION_PATH = f"/home/snfiel01/projects/sad_implementation/directions/llama3.1_8b_base_instruct_directions/{version}/mms_balanced_shared.json"
        output_path = f"outputs/{version}/sae_decomposition_layer{layer}.json"

        # Load SAE for layer 21 residual stream
        print(f"\nLoading SAE for layer {layer} residual stream...")

        print("Loading steering direction...")
        with open(DIRECTION_PATH) as f:
            data = json.load(f)

        direction = torch.tensor(data["shared_directions"][str(layer)], device=DEVICE)

        print(f"Direction shape: {direction.shape}, norm: {direction.norm().item():.4f}")

        # Load SAE for layer residual stream
        print(f"\nLoading SAE for layer {layer} residual stream...")

        sae, cfg_dict, sparsity = SAE.from_pretrained(
            release="llama_scope_lxr_8x",  # r = residual stream
            sae_id=f"l{layer}r_8x",        # layer 21 residual
            device=DEVICE
        )

                
        print(f"SAE config: {cfg_dict.get('d_sae', 'unknown')} features")

        # Get decoder weights (features -> residual stream)
        # Shape: (n_features, d_model)
        W_dec = sae.W_dec.detach()
        print(f"Decoder shape: {W_dec.shape}")

        # Normalize decoder directions
        W_dec_normalized = W_dec / W_dec.norm(dim=1, keepdim=True)

        # Compute cosine similarity between each SAE feature and steering direction
        # direction: (d_model,), W_dec_normalized: (n_features, d_model)
        cosine_sims = W_dec_normalized @ direction

        # Compute stats
        cos_min = cosine_sims.min().item()
        cos_max = cosine_sims.max().item()
        cos_mean = cosine_sims.mean().item()
        cos_std = cosine_sims.std().item()

        print(f"\nCosine similarity stats:")
        print(f"  Min: {cos_min:.4f}")
        print(f"  Max: {cos_max:.4f}")
        print(f"  Mean: {cos_mean:.4f}")
        print(f"  Std: {cos_std:.4f}")

        # Get top K most aligned features (positive alignment = same direction as self)
        top_positive_indices = torch.topk(cosine_sims, TOP_K).indices
        top_positive_values = cosine_sims[top_positive_indices]

        # Get top K most anti-aligned features (negative = opposite direction)
        top_negative_indices = torch.topk(-cosine_sims, TOP_K).indices
        top_negative_values = cosine_sims[top_negative_indices]

        print(f"\n{'='*60}")
        print(f"TOP {TOP_K} FEATURES ALIGNED WITH SELF-DIRECTION (Layer {layer})")
        print(f"{'='*60}")
        print(f"{'Rank':<6}{'Feature ID':<12}{'Cosine Sim':<12}{'Neuronpedia URL'}")
        print("-" * 60)
        for i, (idx, sim) in enumerate(zip(top_positive_indices, top_positive_values)):
            idx_val = idx.item()
            sim_val = sim.item()
            url = f"https://www.neuronpedia.org/llama3.1-8b/{layer}-llamascope-res-32k/{idx_val}"
            print(f"{i+1:<6}{idx_val:<12}{sim_val:<12.4f}{url}")

        print(f"\n{'='*60}")
        print(f"TOP {TOP_K} FEATURES ALIGNED WITH OTHER-DIRECTION (layer {layer})")
        print(f"(Negative alignment = opposite to self)")
        print(f"{'='*60}")
        print(f"{'Rank':<6}{'Feature ID':<12}{'Cosine Sim':<12}{'Neuronpedia URL'}")
        print("-" * 60)
        for i, (idx, sim) in enumerate(zip(top_negative_indices, top_negative_values)):
            idx_val = idx.item()
            sim_val = sim.item()
            url = f"https://www.neuronpedia.org/llama3.1-8b/{layer}-llamascope-res-32k/{idx_val}"
            print(f"{i+1:<6}{idx_val:<12}{sim_val:<12.4f}{url}")

        results = {
            "layer": layer,
            "direction_source": DIRECTION_PATH,
            "sae_release": "llama_scope_lxr_8x",
            "cosine_sim_stats": {
                "min": cos_min,
                "max": cos_max,
                "mean": cos_mean,
                "std": cos_std
            },
            "top_self_aligned": [
                {
                    "feature_id": idx.item(),
                    "cosine_sim": sim.item(),
                    "url": f"https://www.neuronpedia.org/llama3.1-8b/{layer}-llamascope-res-32k/{idx.item()}"
                }
                for idx, sim in zip(top_positive_indices, top_positive_values)
            ],
            "top_other_aligned": [
                {
                    "feature_id": idx.item(),
                    "cosine_sim": sim.item(),
                    "url": f"https://www.neuronpedia.org/llama3.1-8b/{layer}-llamascope-res-32k/{idx.item()}"
                }
                for idx, sim in zip(top_negative_indices, top_negative_values)
            ],
        }

        with open(output_path, "w") as f:
            json.dump(results, f, indent=2)
        print(f"\nResults saved to {output_path}")



Using device: cpu
Processing layer 1 for version V1

Loading SAE for layer 1 residual stream...
Loading steering direction...
Direction shape: torch.Size([4096]), norm: 1.0000

Loading SAE for layer 1 residual stream...


/tmp/ipykernel_3849894/1512906786.py:25: DeprecationWarning: Unpacking SAE objects is deprecated. SAE.from_pretrained() now returns only the SAE object. Use SAE.from_pretrained_with_cfg_and_sparsity() to get the config dict and sparsity as well.
  sae, cfg_dict, sparsity = SAE.from_pretrained(


SAE config: 32768 features
Decoder shape: torch.Size([32768, 4096])

Cosine similarity stats:
  Min: -0.2728
  Max: 0.2614
  Mean: 0.0024
  Std: 0.0162

TOP 20 FEATURES ALIGNED WITH SELF-DIRECTION (Layer 1)
Rank  Feature ID  Cosine Sim  Neuronpedia URL
------------------------------------------------------------
1     16858       0.2614      https://www.neuronpedia.org/llama3.1-8b/1-llamascope-res-32k/16858
2     27791       0.2129      https://www.neuronpedia.org/llama3.1-8b/1-llamascope-res-32k/27791
3     24736       0.1926      https://www.neuronpedia.org/llama3.1-8b/1-llamascope-res-32k/24736
4     13320       0.1785      https://www.neuronpedia.org/llama3.1-8b/1-llamascope-res-32k/13320
5     14476       0.1718      https://www.neuronpedia.org/llama3.1-8b/1-llamascope-res-32k/14476
6     7191        0.1567      https://www.neuronpedia.org/llama3.1-8b/1-llamascope-res-32k/7191
7     11674       0.1552      https://www.neuronpedia.org/llama3.1-8b/1-llamascope-res-32k/11674
8     10

hyperparams.json: 0.00B [00:00, ?B/s]

Llama3_1-8B-Base-L9R-8x/checkpoints/fina(…):   0%|          | 0.00/537M [00:00<?, ?B/s]

...

Llama3_1-8B-Base-L23R-8x/checkpoints/fin(…):   0%|          | 0.00/537M [00:00<?, ?B/s]

SAE config: 32768 features
Decoder shape: torch.Size([32768, 4096])

Cosine similarity stats:
  Min: -0.1526
  Max: 0.2444
  Mean: -0.0025
  Std: 0.0186

TOP 20 FEATURES ALIGNED WITH SELF-DIRECTION (Layer 23)
Rank  Feature ID  Cosine Sim  Neuronpedia URL
------------------------------------------------------------
1     30357       0.2444      https://www.neuronpedia.org/llama3.1-8b/23-llamascope-res-32k/30357
2     16250       0.1609      https://www.neuronpedia.org/llama3.1-8b/23-llamascope-res-32k/16250
3     3384        0.1400      https://www.neuronpedia.org/llama3.1-8b/23-llamascope-res-32k/3384
4     25046       0.1299      https://www.neuronpedia.org/llama3.1-8b/23-llamascope-res-32k/25046
5     25161       0.1273      https://www.neuronpedia.org/llama3.1-8b/23-llamascope-res-32k/25161
6     3540        0.1231      https://www.neuronpedia.org/llama3.1-8b/23-llamascope-res-32k/3540
7     25381       0.1129      https://www.neuronpedia.org/llama3.1-8b/23-llamascope-res-32k/25381


hyperparams.json: 0.00B [00:00, ?B/s]

Llama3_1-8B-Base-L24R-8x/checkpoints/fin(…):   0%|          | 0.00/537M [00:00<?, ?B/s]

SAE config: 32768 features
Decoder shape: torch.Size([32768, 4096])

Cosine similarity stats:
  Min: -0.1591
  Max: 0.2475
  Mean: -0.0032
  Std: 0.0183

TOP 20 FEATURES ALIGNED WITH SELF-DIRECTION (Layer 24)
Rank  Feature ID  Cosine Sim  Neuronpedia URL
------------------------------------------------------------
1     2962        0.2475      https://www.neuronpedia.org/llama3.1-8b/24-llamascope-res-32k/2962
2     8222        0.1663      https://www.neuronpedia.org/llama3.1-8b/24-llamascope-res-32k/8222
3     24854       0.1435      https://www.neuronpedia.org/llama3.1-8b/24-llamascope-res-32k/24854
4     22583       0.1344      https://www.neuronpedia.org/llama3.1-8b/24-llamascope-res-32k/22583
5     1731        0.1223      https://www.neuronpedia.org/llama3.1-8b/24-llamascope-res-32k/1731
6     26371       0.1149      https://www.neuronpedia.org/llama3.1-8b/24-llamascope-res-32k/26371
7     32734       0.1140      https://www.neuronpedia.org/llama3.1-8b/24-llamascope-res-32k/32734
8